In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [79]:
postulations_encoded = pd.read_csv('../MapDataAnalysis/MappedData4M.csv')

In [80]:
postulations_encoded.head()

,Unnamed: 0,descripcion,idaviso,idpostulante,nombre_area,nombre_sort,se_postulo,titulo,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,0,0,1111662226,EzpVOLz,Farmacéutica,3.0,1,Cajeros/as- Atencion al cliente,0,0,2,1,3.0,0
1,1,0,1111413600,kPjordV,Telemarketing,3.0,1,Operador telefónico / Retención y Fidelizació...,0,1,2,1,0.0,1
2,2,1,1112401116,pzNk9Rr,Soporte Técnico,1.0,1,Técnico en Reparación de Celulares,1,0,1,1,3.0,0
3,3,0,1112319004,vV3BMz1,Ventas,1.0,1,Asesores Comerciales NEUQUEN,1,0,1,1,4.0,1
4,4,0,1112094756,a4GRrN,Call Center,3.0,1,Representante de Atencion al Cliente/ Banco,2,1,0,1,3.0,0


In [81]:
postulations_encoded= postulations_encoded.drop(['idaviso','idpostulante','Unnamed: 0', 'titulo','estadoEncoded','sexoEncoded','zonaEncoded','nombre_area','nivelLaboralEncoded'],axis=1)

In [82]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column in postulations_encoded.columns:
    if postulations_encoded[column].dtype == type(object):
        postulations_encoded[column] = le.fit_transform(postulations_encoded[column].astype(str))

postulations_encoded.sample(5)

,descripcion,nombre_sort,se_postulo,tipoDeTrabajoEncoded,edadEncoded
1982607,0,1.0,0,0,4.0
2468642,1,1.0,0,0,1.0
3782832,1,2.0,0,0,0.0
1273186,0,3.0,1,1,0.0
325762,1,2.0,1,0,3.0


In [83]:
#Normalizamos los valores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(postulations_encoded) 
postulations_encoded.loc[:,:] = scaled_values

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(postulations_encoded.drop(['se_postulo'], axis=1).values, postulations_encoded['se_postulo'], test_size=0.20, random_state=75)

In [85]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1,n_estimators=30)

In [86]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [87]:
clf.predict(X_test)

array([0., 0., 0., ..., 0., 0., 0.])

In [88]:
clf.score(X_test, y_test)

0.561503102157695

In [95]:
test = pd.read_csv('../MapDataAnalysis/MappedTest.csv')

In [96]:
test.head()

,id,idaviso,idpostulante,nombre_sort,titulo,descripcion,nombre_area,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,0,739260,6M9ZQR,6.0,Asistente Comercial,1,Comercial,1,0,3,0,4.0,1.0
1,1,739260,6v1xdL,2.0,Asistente Comercial,1,Comercial,1,0,3,1,3.0,1.0
2,2,739260,ezRKm9,3.0,Asistente Comercial,1,Comercial,1,0,3,0,3.0,1.0
3,3,758580,1Q35ej,4.0,Oracle Financials Sr. Application Developer,0,Tecnologia / Sistemas,1,0,2,1,4.0,1.0
4,4,758580,EAN4J6,3.0,Oracle Financials Sr. Application Developer,0,Tecnologia / Sistemas,2,0,2,0,3.0,1.0


In [97]:
test = test.drop(['idaviso','idpostulante','titulo','estadoEncoded','sexoEncoded','zonaEncoded','nombre_area','nivelLaboralEncoded','descripcion'],axis=1)

In [98]:
le = preprocessing.LabelEncoder()
for column in test.columns:
    if test[column].dtype == type(object):
        test[column] = le.fit_transform(test[column].astype(str))

In [99]:
#Normalizamos los valores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(test.drop(columns = ['id'])) 
test.loc[:,1:] = scaled_values

In [100]:
test.head()

,id,nombre_sort,descripcion,tipoDeTrabajoEncoded,edadEncoded
0,0,1.000000,1.0,0.0,1.00
1,1,0.333333,1.0,0.0,0.75
2,2,0.500000,1.0,0.0,0.75
3,3,0.666667,0.0,0.0,1.00
4,4,0.500000,0.0,0.0,0.75


In [101]:
result = clf.predict_proba(test.drop(['id'],axis = 1))

In [102]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [103]:
test['sepostulo'] = result2

In [104]:
test.head()

,id,nombre_sort,descripcion,tipoDeTrabajoEncoded,edadEncoded,sepostulo
0,0,1.000000,1.0,0.0,1.00,0.438221
1,1,0.333333,1.0,0.0,0.75,0.443261
2,2,0.500000,1.0,0.0,0.75,0.443261
3,3,0.666667,0.0,0.0,1.00,0.300718
4,4,0.500000,0.0,0.0,0.75,0.410722


In [105]:
test = test[['id','sepostulo']]

In [106]:
test.head()

,id,sepostulo
0,0,0.438221
1,1,0.443261
2,2,0.443261
3,3,0.300718
4,4,0.410722


In [107]:
FirstRandomForest = test.to_csv('../Predictions/RandromForestResult4MLessFeatures4.csv', index=False)

In [21]:
test['sepostulo'].value_counts()

0    63501
1    36499
Name: sepostulo, dtype: int64